## I. Basics

** Series **

* [DEF] A *sequence* $\{a_n\}$ converges if $lim_{n\rightarrow\infty}a_n = L$ exists. It otherwise diverges.
* [DEF] Let $\sum_{n=1}^\infty a_n = a_1 + ...$, and $s_n=\sum_{i=1}^n a_i = a_1 + ... + a_n$, then $\sum_{n=1}^\infty a_n$ is convergent if $lim_{n\rightarrow\infty}s_n=s$ exists as a real number. Then $\sum_{n=1}^\infty a_n = s$. $s$ is the *sum of series*.
* [DEF] A *series* (the sum of an infinite sequence) $\sum a_n$ is *absolutely convergent* if the series of absolute values $\sum|a_n|$, it is *conditionally convergent* if it is convergent but not absolutely convergent.


* [THM] If $lim_{n\rightarrow\infty}f(x)=L$ and $f(n)=a_n$ when $n$ is an integer, then $lim_{n\rightarrow\infty}a_n=L$.
* [THM] If $lim_{n\rightarrow\infty}a_n=L$ and the function $f$ is continuous at $L$, then $lim_{n\rightarrow\infty}f(a_n) = f(L)$.
* [THM] Every bounded, monotonic sequence is convergent.
* [THM] If $\sum_{n=1}^\infty a_n$ is convergent, then $lim_{n\rightarrow\infty}a_n =0$.
* [THM] If $\sum_{n=1}^\infty a_n$ and $\sum_{n=1}^\infty b_n$ are convergent, so are $c\sum_{n=1}^\infty a_n$, $\sum_{n=1}^\infty (a_n+b_n)$ and $\sum_{n=1}^\infty (a_n-b_n)$.
* [THM] If a series $\sum a_n$ is abolustely convergent, then it is convergent.


* [EXP] $\sum_{n=1}^\infty \frac{1}{n(n+1)} = 1$ (pf. 732).
* [EXP] *Geometric Series*: $\sum_{n=1}^\infty ar^{n-1} = a + ar + ar^2 + ...$. It is convergent if $|r| < 1$ and its sum is $\sum_{n=1}^\infty ar^{n-1} = \frac{a}{1-r},|r|<1$ (pf. 730).
* [EXP] *p-Series*: $\sum_{n=1}^\infty \frac{1}{n^p}$ is convergent if $p > 1$, divergent otherwise.


* [TST] *Test for Divergence*: If $lim_{n\rightarrow\infty}a_n$ does not exist or if $lim_{n\rightarrow\infty}\neq 0$, then the series $\sum_{n=1}^\infty a_n$ is divergent.
* [TST] *Integral Test*: Let $f$ be continuous, positive, decreasing on $[1,\infty)$ and let $a_n=f(n)$, then the series $\sum_{n=1}^\infty a_n$ is,
    * convergent if $\int_1^\infty f(x)dx$ is convergent,
    * divergent if $\int_1^\infty f(x)dx$ is divergent.
    * *Remainder*: $\int_{n+1}^\infty f(x)dx \leq R_n \leq \int_n^\infty f(x)dx$ (grpf. 742), which implies $s_n + \int_{n+1}^\infty f(x)dx \leq s \leq s_n + \int_n^\infty f(x)dx$, since $R_n = s - s_n$.
* [TST] *Comparison Test*: Let $\sum a_n$ and $\sum b_n$ be series with positive terms,
    * If $\sum b_n$ is convergent and $a_n\leq b_n$ for all $n$, then $\sum a_n$ is also convergent,
    * If $\sum b_n$ is divergent and $a_n\geq b_n$ for all $n$, then $\sum a_n$ is also divergent.
* [TST] *Limit Comparsion Test*: Let $\sum a_n$ and $\sum b_n$ be series with positive terms. If $lim_{n\rightarrow\infty}\frac{a_n}{b_n} = c$, where $c$ is a finite number and $c>0$, then either both series converge or both diverge.
* [TST] *Alternating Series Test*: Given the series $\sum_{n=1}^\infty (-1)^{n-1} b_n = b_1-b_2+b_3-b_4+b_5-b_6+...,b_n>0$, if 
    * $b_{n+1}\leq b_n,\forall n$,
    * $lim_{n\rightarrow\infty}b_n=0$.
    
    then the series of convergent.
* [TST] *Ratio Test*:
    * If $lim_{n\rightarrow\infty}\left|\frac{a_{n+1}}{a_n}\right| = L < 1$, then the series $\sum_{n=1}^\infty a_n$ is absolutely convergent.
    * If $lim_{n\rightarrow\infty}\left|\frac{a_{n+1}}{a_n}\right| = L > 1$ or $lim_{n\rightarrow\infty}\left|\frac{a_{n+1}}{a_n}\right| = \infty$, then the series $\sum_{n=1}^\infty a_n$ is divergent.
    * If $lim_{n\rightarrow\infty}\left|\frac{a_{n+1}}{a_n}\right| = 1$, then the test is inconclusive.
* [TST] *Root Test*:
    * If $lim_{n\rightarrow\infty}\sqrt[n]{|a_n|} = L < 1$, then the series $\sum_{n=1}^\infty a_n$ is absolutely convergent.
    * If $lim_{n\rightarrow\infty}\sqrt[n]{|a_n|} = L > 1$ or $lim_{n\rightarrow\infty}\sqrt[n]{|a_n|} = \infty$, then the series $\sum_{n=1}^\infty a_n$ is divergent.
    * If $lim_{n\rightarrow\infty}\sqrt[n]{|a_n|} = 1$, then the test is inconclusive.